In [1]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value
import numpy as np
import pandas as pd
from scipy import stats


In [2]:
symbol="V"
timeInterval = "1d"
timeRange = "1y"

In [3]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T',
       'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC',
       'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW',
       'COG', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR',
       'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN',
       'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF',
       'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO',
       'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO',
 

In [4]:
my_columns = ["Ticker", "Stock Price","4 Week Change", "4 Week Percentile", "12 Week Change", "12 Week Percentile", "26 Week Change", "26 Week Percentile", "52 Week Change", "52 Week Percentile", "HQM Score"]
dataframe = pd.DataFrame(columns=my_columns)
for sym in sp500symbols[:]:
    try: 
        if(sym.find(".") > -1):
            sym = sym.replace(".", "-")
        basic = basic_stats(sym)
        complexStats = extended_stock_stats(sym)
        previousClose = parse_value(basic["Previous Close"])
        metadataAndPrice = metadata_price_action(sym, "1d", "1y")
        if(len(metadataAndPrice["price"]) >= 250):
            dataLength = len(metadataAndPrice["price"])
            fourWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-19]["close"])/metadataAndPrice["price"].iloc[dataLength-19]["close"]
            twelveWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-59]["close"])/metadataAndPrice["price"].iloc[dataLength-59]["close"]
            twentySixWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-129]["close"])/metadataAndPrice["price"].iloc[dataLength-129]["close"]
            fiftyTwoWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
            dataframe = dataframe.append(pd.Series([sym,previousClose,fourWeekChange, "N/A", twelveWeekChange, "N/A", twentySixWeekChange, "N/A", fiftyTwoWeekChange, "N/A", "N/A"], index=my_columns), ignore_index=True)
            print(sym, previousClose, fourWeekChange, twelveWeekChange, twentySixWeekChange, fiftyTwoWeekChange)
    except:
        print(f'Error processing data for symbol: {sym}')
# dataframe.sort_values("52 Week Change", ascending=False, inplace=True)
dataframe.reset_index(inplace=True)
dataframe.to_csv("lastBetterMomentumDataframe_bak.csv")
# dataframe = dataframe[:50]
dataframe

MMM 202.4 0.0335929458060193 0.13637387124877778 0.24628700307797882 0.36738276772928846
ABT 118.7 -0.033005488313803365 -0.07236995311935329 0.1067174658826307 0.23815964740396092
ABBV 116.0 0.06958047213416906 0.10829181629844344 0.311614730878187 0.31683729227151025
ABMD 295.7 -0.10049399502814207 -0.09608353392752586 0.16464583264728158 0.5866252522467703
ACN 291.1 0.02212326460832975 0.13228724665829142 0.3239928949856594 0.5460861399762476
ATVI 93.0 -0.02809101740877011 -0.08679312897344403 0.24083771149724767 0.2961444331810445
ADBE 483.6 -0.050751612299782846 -0.020266990268988924 0.098417782857563 0.3298413594176923
AMD 77.8 -0.01720918868684224 -0.15953929423094243 0.05502009073975307 0.48166947910052893
AAP 204.1 0.09103034643418528 0.3380669098060863 0.350706794989134 0.6235508754720968
AES 25.4 -0.08675633411642243 -0.09603924658861714 0.2895000457763672 1.0798388474937823
AFL 56.1 0.07305411413159647 0.19586536231983034 0.5796734202487178 0.547435278755386
A 132.4 0.00699

,index,Ticker,Stock Price,4 Week Change,4 Week Percentile,12 Week Change,12 Week Percentile,26 Week Change,26 Week Percentile,52 Week Change,52 Week Percentile,HQM Score
0,0,MMM,202.4,0.033593,N/A,0.136374,N/A,0.246287,N/A,0.367383,N/A,N/A
1,1,ABT,118.7,-0.033005,N/A,-0.072370,N/A,0.106717,N/A,0.238160,N/A,N/A
2,2,ABBV,116.0,0.069580,N/A,0.108292,N/A,0.311615,N/A,0.316837,N/A,N/A
3,3,ABMD,295.7,-0.100494,N/A,-0.096084,N/A,0.164646,N/A,0.586625,N/A,N/A
4,4,ACN,291.1,0.022123,N/A,0.132287,N/A,0.323993,N/A,0.546086,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,498,YUM,121.2,0.032184,N/A,0.149426,N/A,0.272756,N/A,0.406707,N/A,N/A
499,499,ZBRA,471.4,-0.031671,N/A,0.022690,N/A,0.667110,N/A,1.011279,N/A,N/A
500,500,ZBH,173.2,0.023862,N/A,0.068844,N/A,0.301615,N/A,0.415477,N/A,N/A
501,501,ZION,59.1,0.067351,N/A,0.195442,N/A,0.784166,N/A,0.929362,N/A,N/A


In [5]:
dataframe.to_csv("momentum_almost.csv")

In [5]:
time_periods = [
                '52 Week',
                '26 Week',
                '12 Week',
                '4 Week'
                ]

for row in dataframe.index:
    for time_period in time_periods:
        dataframe.loc[row, f'{time_period} Percentile'] = stats.percentileofscore(dataframe[f'{time_period} Change'], dataframe.loc[row, f'{time_period} Change'])/100

from statistics import mean

for row in dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(dataframe.loc[row, f'{time_period} Percentile'])
    dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
sorted_dataframe = dataframe.sort_values(by = 'HQM Score', ascending = False)
sorted_dataframe.to_csv("momentum_last.csv")
sorted_dataframe

,index,Ticker,Stock Price,4 Week Change,4 Week Percentile,12 Week Change,12 Week Percentile,26 Week Change,26 Week Percentile,52 Week Change,52 Week Percentile,HQM Score
204,204,FCX,42.0,0.303587,0.998012,0.407941,0.99006,1.410636,0.99006,3.896437,0.996024,0.993539
346,346,NUE,95.6,0.261625,0.994036,0.752929,1,1.020872,0.960239,1.340284,0.924453,0.969682
84,84,COF,156.4,0.188739,0.968191,0.339363,0.962227,1.083906,0.976143,1.535531,0.954274,0.965209
318,318,MHK,224.8,0.134015,0.916501,0.357160,0.976143,1.003139,0.956262,1.839802,0.972167,0.955268
140,140,DVN,25.2,0.208852,0.974155,0.312312,0.940358,1.780488,0.994036,1.104334,0.862823,0.942843
...,...,...,...,...,...,...,...,...,...,...,...,...
64,64,BDX,240.8,-0.045114,0.0954274,-0.066050,0.0914513,0.019592,0.0775348,-0.086472,0.027833,0.0730616
345,345,NRG,34.7,-0.101344,0.0337972,-0.192379,0.027833,0.080510,0.151093,0.060726,0.0735586,0.0715706
361,361,PAYC,327.1,-0.159426,0.00994036,-0.206790,0.0238569,-0.091570,0.00795229,0.204205,0.208748,0.0626243
111,111,CTXS,121.6,-0.129281,0.0159046,-0.120933,0.0536779,0.057677,0.125249,-0.214193,0.00397614,0.0497018


In [16]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'V',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 1205933400,
 'regularMarketTime': 1614373202,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 212.39,
 'chartPreviousClose': 187.21,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '1y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [17]:
price = metadataAndPrice["price"]
price

,timestamp,high,volume,close,open,low
0,1582813800,186.339996,17536600,180.009995,182.119995,180.000000
1,1582900200,182.330002,22674700,181.759995,173.789993,172.979996
2,1583159400,192.440002,17434500,192.330002,186.320007,183.100006
3,1583245800,194.039993,15226400,185.729996,193.199997,184.500000
4,1583332200,194.490005,10789100,194.289993,190.360001,186.169998
...,...,...,...,...,...,...
248,1614004200,210.649994,8332700,208.320007,203.550003,203.110001
249,1614090600,213.910004,10568500,212.110001,207.399994,206.550003
250,1614177000,220.529999,12005100,219.429993,212.360001,211.289993
251,1614263400,220.449997,11400200,213.750000,219.289993,212.580002


In [18]:
basicStats = basic_stats(symbol)
basicStats

STATS <Response [200]>


{'Previous Close': '213.75',
 'Open': '214.05',
 'Bid': '212.01 x 900',
 'Ask': '212.74 x 3100',
 "Day's Range": '210.74 - 215.21',
 '52 Week Range': '133.93 - 220.53',
 'Volume': '11,730,004',
 'Avg. Volume': '9,401,263',
 'Market Cap': '468.288B',
 'Beta (5Y Monthly)': '0.98',
 'PE Ratio (TTM)': '43.85',
 'EPS (TTM)': '4.84',
 'Earnings Date': 'Apr 28, 2021 - May 03, 2021',
 'Forward Dividend & Yield': '1.28 (0.60%)',
 'Ex-Dividend Date': 'Feb 11, 2021'}

In [19]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['483.81B',
  '482.33B',
  '438.37B',
  '424.23B',
  '354.81B',
  '416.79B'],
 'Enterprise Value 3': ['490.51B',
  '491.45B',
  '447.91B',
  '435.44B',
  '364.29B',
  '426.91B'],
 'Trailing P/E ': ['45.24', '44.73', '38.02', '34.74', '29.40', '35.32'],
 'Forward P/E 1': ['40.65', '39.84', '34.25', '32.15', '28.74', '30.12'],
 'PEG Ratio (5 yr expected) 1': ['1.88',
  '3.69',
  '2.35',
  '2.37',
  '2.01',
  '2.06'],
 'Price/Sales (ttm)': ['22.61', '22.26', '19.52', '18.17', '15.47', '18.58'],
 'Price/Book (mrq)': ['13.80', '15.50', '14.06', '14.12', '11.90', '14.26'],
 'Enterprise Value/Revenue 3': ['22.84',
  '86.42',
  '87.81',
  '90.02',
  '62.23',
  '70.52'],
 'Enterprise Value/EBITDA 6': ['32.95',
  '120.42',
  '131.89',
  '133.12',
  '88.01',
  '101.69'],
 'Beta (5Y Monthly) ': '0.98',
 '52-Week Change 3': '11.14%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '220.53',
 '52 Week Low 3': '133.93',
 '50-Day Moving Average 3': '206.01',
 '200-